# Philosopher Reasearch

## Sentiment Analysis

This code block is importing necessary Python libraries

In [13]:
# importing required libraries
import numpy as np 
import pandas as pd 
import os
from dotenv import load_dotenv
load_dotenv()
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation

Find the directory path of data

In [14]:
for dirname, _, filenames in os.walk(os.getenv('data_path')):
    for filename in filenames:
        main_path = os.path.join(dirname, filename)

Read in the csv data and save it to a dafa frame

In [15]:
df = pd.read_csv(main_path)
df.head()

,title,author,school,sentence_spacy,sentence_str,original_publication_date,corpus_edition_date,sentence_length,sentence_lowered,tokenized_txt,lemmatized_str
0,Plato - Complete Works,Plato,plato,"What's new, Socrates, to make you leave your ...","What's new, Socrates, to make you leave your ...",-350,1997,125,"what's new, socrates, to make you leave your ...","['what', 'new', 'socrates', 'to', 'make', 'you...","what be new , Socrates , to make -PRON- lea..."
1,Plato - Complete Works,Plato,plato,Surely you are not prosecuting anyone before t...,Surely you are not prosecuting anyone before t...,-350,1997,69,surely you are not prosecuting anyone before t...,"['surely', 'you', 'are', 'not', 'prosecuting',...",surely -PRON- be not prosecute anyone before ...
2,Plato - Complete Works,Plato,plato,The Athenians do not call this a prosecution b...,The Athenians do not call this a prosecution b...,-350,1997,74,the athenians do not call this a prosecution b...,"['the', 'athenians', 'do', 'not', 'call', 'thi...",the Athenians do not call this a prosecution ...
3,Plato - Complete Works,Plato,plato,What is this you say?,What is this you say?,-350,1997,21,what is this you say?,"['what', 'is', 'this', 'you', 'say']",what be this -PRON- say ?
4,Plato - Complete Works,Plato,plato,"Someone must have indicted you, for you are no...","Someone must have indicted you, for you are no...",-350,1997,101,"someone must have indicted you, for you are no...","['someone', 'must', 'have', 'indicted', 'you',...","someone must have indict -PRON- , for -PRON- ..."


In [16]:
stopwords = list(STOP_WORDS)

class DataSummarizer:

    # initialise variables
    def __init__(self, path = None, corpus = None):
        self.path = path
        self.corpus = corpus
        self.wordFrequencies = {}
        self.sentimentScore = {}
        self.numSentences = 0
        self.nlp = spacy.load('en_core_web_sm')
        self.doc = self.nlp(self.corpus)

    def printCorpus(self):
        print(self.corpus)

    def isWordInDict(self, word):
        if word not in self.wordFrequencies.keys():
            return False
        else:
            return True

    def calculateWordFrequency(self):
        for word in self.doc:
            lowerWord = word.text.lower()
            if self.isWordInDict(self, lowerWord):
                self.wordFrequencies[lowerWord] += 1
            else:
                self.wordFrequencies[lowerWord] = 1

